# Neural Time Series Forescasting

In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as  np
import seaborn as sns #estadisticas basicas de visualizacion

sns.set(style="whitegrid", color_codes=True)

path='../2018_01_23_datos_walmart/'
df_train = pd.read_csv(path +'1_data.csv', index_col=0)

#sacar todos los Item que no sean 3,4,14,66,70,168,173,316,415,475
items=[3,4,14,40,66,70,168,173,316,415,475]
#df = df[df.Item==1]
df_train=df_train[df_train['Item'].isin(items)]#los que se encuentran en el arreglo


df_train.index = pd.to_datetime(df_train.index)
df_train.shape #dimensionalidad

df_train.columns.values.tolist()


['Local', 'Item', 'Unidades', 'Venta', 'Inventario']

De manera de utilizar redes neuronales, debemos convertir las series de tiempo en un problema de aprendizaje supervisado. 

In [52]:
def series_to_supervised(data, window=1, lag=1, dropnan=True):
    cols, names = list(), list()
    # Input sequence (t-n, ... t-1)
    for i in range(window, 0, -1):
        cols.append(data.shift(i))
        names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    # Current timestep (t=0)
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    # Target timestep (t=lag)
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    # Put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # Drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

window = 1 # t - windows
lag = 1 # t + lag
df_train = series_to_supervised(df_train, window=window, lag=lag)
df_train.columns.values.tolist()

['Local(t-1)',
 'Item(t-1)',
 'Unidades(t-1)',
 'Venta(t-1)',
 'Inventario(t-1)',
 'Local(t)',
 'Item(t)',
 'Unidades(t)',
 'Venta(t)',
 'Inventario(t)',
 'Local(t+1)',
 'Item(t+1)',
 'Unidades(t+1)',
 'Venta(t+1)',
 'Inventario(t+1)']

In [59]:
df_train.head()

,Unidades(t-1),Venta(t-1),Inventario(t-1),Local,Item,Unidades(t),Venta(t),Inventario(t),Unidades(t+1),Venta(t+1),Inventario(t+1),weekend
Fecha,,,,,,,,,,,,
2017-01-07,1.0,6714.0,76.0,75,173,2.0,13428.0,18.0,1.0,6714.0,82.0,1
2017-01-07,3.0,20142.0,61.0,69,173,3.0,20142.0,36.0,2.0,13428.0,90.0,1
2017-01-07,3.0,20142.0,36.0,69,14,2.0,13428.0,90.0,1.0,6714.0,68.0,1
2017-01-07,1.0,6714.0,70.0,82,173,1.0,6714.0,25.0,2.0,13428.0,89.0,1
2017-01-07,5.0,33570.0,32.0,58,14,1.0,6714.0,100.0,1.0,6714.0,22.0,1


In [60]:
print df_train.shape

(9301, 12)


In [55]:
print df_train.columns

Index([u'Local(t-1)', u'Item(t-1)', u'Unidades(t-1)', u'Venta(t-1)',
       u'Inventario(t-1)', u'Local(t)', u'Item(t)', u'Unidades(t)',
       u'Venta(t)', u'Inventario(t)', u'Local(t+1)', u'Item(t+1)',
       u'Unidades(t+1)', u'Venta(t+1)', u'Inventario(t+1)'],
      dtype='object')


In [56]:
#barramos todos los valores que no son variables temporales, en este caso Item y Local

columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['Item', 'Local']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['Item', 'Local']]
    
print columns_to_drop

['Item(t+1)', 'Local(t+1)', 'Item(t-1)', 'Local(t-1)']


In [57]:
#ejecutar borrado de columnas
df_train.drop(labels=columns_to_drop, inplace=True, axis=1)
df_train.rename({'Local(t)':'Local', 'Item(t)':'Item'}, inplace=True,axis='columns')
df_train.tail()

,Unidades(t-1),Venta(t-1),Inventario(t-1),Local,Item,Unidades(t),Venta(t),Inventario(t),Unidades(t+1),Venta(t+1),Inventario(t+1)
Fecha,,,,,,,,,,,
2018-03-31,1.0,3353.0,17.0,19,70,3.0,12579.0,44.0,4.0,33580.0,2.0
2018-03-31,3.0,12579.0,44.0,19,66,4.0,33580.0,2.0,5.0,33570.0,103.0
2018-03-31,2.0,6706.0,18.0,45,40,2.0,6706.0,18.0,5.0,20965.0,1.0
2018-03-31,1.0,6714.0,7.0,28,14,1.0,5042.0,32.0,1.0,3353.0,21.0
2018-03-31,1.0,5042.0,32.0,28,415,1.0,3353.0,21.0,2.0,16790.0,46.0


Incorporamos información foránea al modelo. En este caso, convertimos la fecha en dia, mes, año y día de la semana.

In [58]:
def expand_df(df):
    data = df.copy()
    #data['month'] = data.index.month
    #data['year'] = data.index.year
    data['weekend'] = np.int32(data.index.dayofweek > 3)
    return data

df_train=expand_df(df_train)
df_train.tail()

,Unidades(t-1),Venta(t-1),Inventario(t-1),Local,Item,Unidades(t),Venta(t),Inventario(t),Unidades(t+1),Venta(t+1),Inventario(t+1),weekend
Fecha,,,,,,,,,,,,
2018-03-31,1.0,3353.0,17.0,19,70,3.0,12579.0,44.0,4.0,33580.0,2.0,1
2018-03-31,3.0,12579.0,44.0,19,66,4.0,33580.0,2.0,5.0,33570.0,103.0,1
2018-03-31,2.0,6706.0,18.0,45,40,2.0,6706.0,18.0,5.0,20965.0,1.0,1
2018-03-31,1.0,6714.0,7.0,28,14,1.0,5042.0,32.0,1.0,3353.0,21.0,1
2018-03-31,1.0,5042.0,32.0,28,415,1.0,3353.0,21.0,2.0,16790.0,46.0,1


A continuacion, separamos la variable dependientes $y$ de las independentes $X$. En este caso, la etiqueta corresponde a la columna : sales(t+1)

In [9]:
labels_col = 'Venta(t+%d)' % lag
X = df_train.drop(labels_col, axis=1)
y = df_train[labels_col]

Transformamos las etiquetas store e item en variables categoricas

In [10]:
from sklearn.preprocessing import OneHotEncoder

Local_ohe = OneHotEncoder()
Item_ohe = OneHotEncoder()

X_Local = pd.DataFrame(Local_ohe.fit_transform(X.Local.values.reshape(-1,1)).toarray())
X_Item = pd.DataFrame(Item_ohe.fit_transform(X.Item.values.reshape(-1,1)).toarray())

X = X.drop(['Local','Item'], axis=1)

X_Local.columns=[u+str(v) for u,v in zip(['Local_']*88,range(1,89))]
X_Item.columns=[u+str(v) for u,v in zip(['Item_']*447,range(1,448))]



/home/pc/env/local/lib/python2.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [11]:
X = np.concatenate([X,X_Item,X_Local],axis=1)
y = y.values

print X.shape
print y.shape

(128234, 544)
(128234,)


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_test.shape)

# convertir a tensor
X_train=X_train.reshape((X_train.shape[0],1,X_train.shape[1]))
X_test=X_test.reshape((X_test.shape[0],1,X_test.shape[1]))
print X_train.shape

('Train set shape', (102587, 544))
('Validation set shape', (25647, 544))
(102587, 1, 544)


# LSTM Model Training

In [13]:
import keras   # Keras es una biblioteca de Redes Neuronalescapaz de ejecutarse sobre TensorFlow
from keras.layers import Dense #Una capa densa es solo una capa regular de neuronas en una red neuronal. recibe información de todas las neuronas en la capa anterior, por lo tanto, está densamente conectada.
from keras.models import Sequential#El modelo secuencial es una pila lineal de capas.
from keras.utils import np_utils
import itertools
from keras.layers import LSTM
from keras import optimizers
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

epochs = 40
batch_size = 256
lr = 0.0003

model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mse',optimizer="rmsprop", metrics=['mape'])
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50)                119000    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 51        
Total params: 119,051
Trainable params: 119,051
Non-trainable params: 0
_________________________________________________________________


In [14]:
history = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=epochs, verbose=1)

Train on 102587 samples, validate on 25647 samples
Epoch 1/40
102587/102587 [==============================] - 9s 89us/step - loss: 272743135.2972 - mean_absolute_percentage_error: 126.7250 - val_loss: 274887057.7050 - val_mean_absolute_percentage_error: 134.9661
Epoch 2/40
102587/102587 [==============================] - 8s 77us/step - loss: 261742238.8636 - mean_absolute_percentage_error: 134.2325 - val_loss: 269200863.3375 - val_mean_absolute_percentage_error: 130.7143
Epoch 3/40
102587/102587 [==============================] - 8s 76us/step - loss: 257453275.6334 - mean_absolute_percentage_error: 139.2179 - val_loss: 266488835.2615 - val_mean_absolute_percentage_error: 145.9140
Epoch 4/40
102587/102587 [==============================] - 8s 77us/step - loss: 254691188.1858 - mean_absolute_percentage_error: 141.6325 - val_loss: 263681223.0695 - val_mean_absolute_percentage_error: 138.4552
Epoch 5/40
102587/102587 [==============================] - 8s 78us/step - loss: 252678321.0798 -

102587/102587 [==============================] - 8s 75us/step - loss: 167636618.1344 - mean_absolute_percentage_error: 109.1050 - val_loss: 174921781.6253 - val_mean_absolute_percentage_error: 95.4168
Epoch 40/40
102587/102587 [==============================] - 8s 75us/step - loss: 167199912.0625 - mean_absolute_percentage_error: 108.0951 - val_loss: 173767676.6212 - val_mean_absolute_percentage_error: 103.4522


In [16]:
model.save('lstm_continuous_input.h5')